In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('../testtask/test_data.csv')
df.head()

,dlg_id,line_n,role,text
0,0,0,client,Алло
1,0,1,manager,Алло здравствуйте
2,0,2,client,Добрый день
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...
4,0,4,client,Ага


### Ознакомление с данными
Видим, что у нас было 5 диалогов. (dig_id - номер диалога)
<br>Следующий столбик - реплика каждого участника (line_n)
<br>Оставим только ответы менеджера и сгруппируем наш датафрейм по номеру диалога

In [3]:
df_new = df[df.role == 'manager'].groupby(['dlg_id', 'line_n'], as_index=False)['text'].sum()

In [4]:
df_new

,dlg_id,line_n,text
0,0,1,Алло здравствуйте
1,0,3,Меня зовут ангелина компания диджитал бизнес з...
2,0,5,Угу ну возможно вы рассмотрите и другие вариан...
3,0,8,Угу а на что вы обращаете внимание при выборе
4,0,11,Что для вас приоритет
...,...,...,...
196,5,135,Так дмитрий
197,5,136,Все записала тогда завтра ждите звонка
198,5,138,По поводу виджетов и с ними уже обсудите конкр...
199,5,139,Все я вам высылаю счет и с вами на связи если ...


In [5]:
#используем для того чтоб посмотреть на токены
from yargy.tokenizer import MorphTokenizer

In [6]:
TOKENIZER = MorphTokenizer()

In [7]:
list(TOKENIZER('компания китобизнес'))

[MorphToken(
     value='компания',
     span=[0, 8),
     type='RU',
     forms=[Form('компания', Grams(NOUN,femn,inan,nomn,sing))]
 ),
 MorphToken(
     value='китобизнес',
     span=[9, 19),
     type='RU',
     forms=[Form('китобизнес', Grams(NOUN,inan,masc,nomn,sing)),
      Form('китобизнес', Grams(NOUN,accs,inan,masc,sing))]
 )]

### Приветствие
Найдем из всех реплик реплики с приветствием

In [15]:
#создаем свои правила для нахождения нужных нам слов и с помощью метода Parser найдем их в тексте
from yargy import rule, Parser, or_, and_, not_
from yargy.predicates import eq, gram
from yargy.pipelines import morph_pipeline
from yargy.interpretation import fact

HELLO = morph_pipeline([
                        'Добрый день',
                        'Здравствуйте'
                        ])

name_finder = rule(HELLO)
parser_name = Parser(name_finder)

found = []
index_found = []
for num, row in zip(df_new.dlg_id, df_new.text):
    matches = (list(parser_name.findall(row)))
    if matches:
        found.append(row)
        index_found.append(num)

In [16]:
greetings = pd.DataFrame(data={'dlg_id':index_found, 'greetings':found})
greetings.head()

,dlg_id,greetings
0,0,Алло здравствуйте
1,1,Алло здравствуйте
2,2,Алло здравствуйте
3,3,Алло дмитрий добрый день


## Прощание
Найдем из всех реплик реплики с прощанием

In [17]:
BYE = morph_pipeline([
                         'До свидания'

                         ])
name_finder = rule(BYE)
parser_name = Parser(name_finder)

found1 = []
index_found1 = []
for num, row in zip(df_new.dlg_id, df_new.text):
    matches = (list(parser_name.findall(row)))
    if matches:
        found1.append(row)
        index_found1.append(num)

In [18]:
parting = pd.DataFrame(data={'dlg_id':index_found1, 'parting':found1})
parting.head()

,dlg_id,parting
0,0,Всего хорошего до свидания
1,1,До свидания
2,4,Во вторник все ну с вами да тогда до вторника ...
3,5,Ну до свидания хорошего вечера


In [19]:
df_full = greetings.merge(parting, how='outer', on='dlg_id')

### Приветствие и прощание
Найдем менеджера, который совершил оба действия

In [20]:
df_full['boolean_greetings'] = df_full.dlg_id.apply(lambda x: True if x in index_found else False)
df_full['boolean_parting'] = df_full.dlg_id.apply(lambda x: True if x in index_found1 else False)
df_full.head()

,dlg_id,greetings,parting,boolean_greetings,boolean_parting
0,0,Алло здравствуйте,Всего хорошего до свидания,True,True
1,1,Алло здравствуйте,До свидания,True,True
2,2,Алло здравствуйте,NaN,True,False
3,3,Алло дмитрий добрый день,NaN,True,False
4,4,NaN,Во вторник все ну с вами да тогда до вторника ...,False,True


В столбцах <b>boolean_greetings</b> и <b>boolean_parting</b> мы получили <b>True</b> для тех реплик, где есть приветствие/прощание и <b>False</b>, где нет

In [21]:
df_full.loc[(df_full.boolean_greetings == True) & (df_full.boolean_parting == True)]

,dlg_id,greetings,parting,boolean_greetings,boolean_parting
0,0,Алло здравствуйте,Всего хорошего до свидания,True,True
1,1,Алло здравствуйте,До свидания,True,True


Поздоровался и попрощался менеджер из нулевого и первого диалога

### Имя
Теперь найдем имена менеджеров, заодно узнаем, кто поздоровался

In [22]:
#Из библиотеки Наташа возьмем метод NamesExtractor чтоб вытащить все имена 
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,

    Doc
)


segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

names_extractor = NamesExtractor(morph_vocab)

In [23]:
index_ = []
p_names = []
for num, row in zip(df_new.dlg_id, df_new.text):
    matches = names_extractor(row)
    for i in matches:
            if i.fact.first != None:
                text = i.fact.first
                #добавим слова, которые могут находится перед представлением человека
                context = f'Меня зовут {text}' 
                context2 = f'меня {text} зовут' 
                context3 = f'это {text}'
                if any(word in row for word in (context, context2, context3)):
                    index_.append(num)
                    p_names.append(text.capitalize())

In [24]:
manager_name = pd.DataFrame(data={'dlg_id':index_, 'manager_name':p_names})

In [25]:
df_full = df_full.merge(manager_name, how='outer', on='dlg_id')

In [26]:
df_full.loc[(df_full.boolean_greetings == True) & (df_full.boolean_parting == True)]

,dlg_id,greetings,parting,boolean_greetings,boolean_parting,manager_name
0,0,Алло здравствуйте,Всего хорошего до свидания,True,True,Ангелина
1,1,Алло здравствуйте,До свидания,True,True,Ангелина


Объеденяя по удовлетворяющим нас фразам в диалоге (приветствие/прощание), наблюдаем, что только Ангелина в 0 и 1 диалогах поприветствовала и попрощалась с клиентом

### Название компании
По принципу, похожему на тот, что использовали с приветствием и прощанием, ищем название компании

In [27]:
#создаем правило для парсинга
COMPANY = rule(eq('компания'), or_(gram('Abbr'), gram('NOUN')), eq('бизнес').optional())

company_finder = rule(COMPANY)
parser_company = Parser(company_finder)

companies = []
companies_index = []
for num, row in zip(df_new.dlg_id, df_new.text):
    matches = (list(parser_company.findall(row)))
    for i in matches:
        companies.append([k.value for k in i.tokens])
        companies_index.append(num)

In [28]:
companies = [' '.join(i[1:]) for i in companies]

In [29]:
company_name = pd.DataFrame(data={'dlg_id':companies_index, 'company_name':companies})

In [30]:
company_name

,dlg_id,company_name
0,0,диджитал бизнес
1,1,диджитал бизнес
2,2,диджитал бизнес
3,3,китобизнес


### Финальная таблица

In [31]:
df_full = df_full.merge(company_name, how='outer', on='dlg_id')
df_full

,dlg_id,greetings,parting,boolean_greetings,boolean_parting,manager_name,company_name
0,0,Алло здравствуйте,Всего хорошего до свидания,True,True,Ангелина,диджитал бизнес
1,1,Алло здравствуйте,До свидания,True,True,Ангелина,диджитал бизнес
2,2,Алло здравствуйте,NaN,True,False,Ангелина,диджитал бизнес
3,3,Алло дмитрий добрый день,NaN,True,False,Максим,китобизнес
4,4,NaN,Во вторник все ну с вами да тогда до вторника ...,False,True,NaN,NaN
5,5,NaN,Ну до свидания хорошего вечера,False,True,Анастасия,NaN
